# PyCity Schools Analysis

In [1]:
#import dependencies
import pandas as pd
from pathlib import Path


In [2]:
#create reference to csv files (schools_complete.csv and students_complete.csv)
schools_csv = Path("Resources/schools_complete.csv")
students_csv = Path("Resources/students_complete.csv")

#read files and create dataframes
schools_df = pd.read_csv(schools_csv)
students_df = pd.read_csv(students_csv)

#combine into one dataframe
merged_df = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])
merged_df.head()



,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
merged_df.columns


Index(['Student ID', 'student_name', 'gender', 'grade', 'school_name',
       'reading_score', 'math_score', 'School ID', 'type', 'size', 'budget'],
      dtype='object')

# District Summary

In [4]:
#calculate the total number of unique schools; this block is working

school_count = len(schools_df)
school_count 



15

In [5]:
#calculate the total number of unique students; this block is working

student_count = len(students_df)
student_count


39170

In [6]:
#calculate the total budget; this block is working

total_budget = schools_df["budget"].sum()
total_budget


24649428

In [7]:
#calculate the average math score
avg_math = students_df["math_score"].mean()
avg_math


78.98537145774827

In [8]:
#calculate the average reading score
avg_read = students_df["reading_score"].mean()
avg_read


81.87784018381414

In [9]:
#use the following to calculate the precentage of students who passed math (math scores greater than or equal to 90)
passing_math_count = merged_df[(merged_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [10]:
#calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)
passing_reading_count = merged_df[(merged_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [11]:
#use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = merged_df[
    (merged_df["math_score"] >= 70) & (merged_df["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [12]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
#num of schools, total students, total budget, avg math score, avg reading score, % math pass, % reading pass, overall pass%

district_summary = pd.DataFrame({"Metric": ["Total Schools", "Total Students", "Total Budget", "Average Math Score",
                                           "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"], 
                                 "Value": [school_count, student_count, total_budget, avg_math, avg_read, passing_math_percentage,
                                           passing_reading_percentage, overall_passing_rate]})

#this option works
pd.options.display.float_format = '{:,.0f}'.format


# Display the DataFrame
district_summary


,Metric,Value
0,Total Schools,15
1,Total Students,"39,170"
2,Total Budget,"24,649,428"
3,Average Math Score,79
4,Average Reading Score,82
5,% Passing Math,75
6,% Passing Reading,86
7,% Overall Passing,65


# School Summary

In [95]:
#select all of the school types
schools = merged_df.groupby("school_name")
school_types = schools["type"].unique()

school_types




school_name
Bailey High School       [District]
Cabrera High School       [Charter]
Figueroa High School     [District]
Ford High School         [District]
Griffin High School       [Charter]
Hernandez High School    [District]
Holden High School        [Charter]
Huang High School        [District]
Johnson High School      [District]
Pena High School          [Charter]
Rodriguez High School    [District]
Shelton High School       [Charter]
Thomas High School        [Charter]
Wilson High School        [Charter]
Wright High School        [Charter]
Name: type, dtype: object

In [96]:
#calculate the total student count per school
#may not need this- gives the total student count per school type
school_type_counts = merged_df["type"].value_counts()

#student counts
school_size = merged_df["school_name"].value_counts()
school_size





school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: count, dtype: int64

In [101]:
#calculate the total school budget per capita spending per school
#on school file, this is budget / size (these same columns are in the merged_df)
per_school_budget = merged_df.groupby(["school_name"])["budget"].unique()
school_size = merged_df.groupby(["school_name"])["size"].unique()
per_school_capita = per_school_budget / school_size

per_school_capita = per_school_capita.astype(float)
per_school_capita



school_name
Bailey High School      628
Cabrera High School     582
Figueroa High School    639
Ford High School        644
Griffin High School     625
Hernandez High School   652
Holden High School      581
Huang High School       655
Johnson High School     650
Pena High School        609
Rodriguez High School   637
Shelton High School     600
Thomas High School      638
Wilson High School      578
Wright High School      583
dtype: float64

In [104]:
#calculate average test scores per school
school_group_df = merged_df.set_index(["school_name"])
school_group_df = school_group_df.groupby(["school_name"])
total_math = school_group_df["math_score"].sum()
total_reading = school_group_df["reading_score"].sum()

per_school_math = school_group_df["math_score"].mean()
per_school_reading = school_group_df["reading_score"].mean()




In [105]:
#Calculate the number of students per school with math scores of 70 or higher
#use loc to find those > 70 and do a student count??

#loc to find scores > 70
students_passing_math = merged_df.loc[merged_df["math_score"] >= 70, :]

#group them by school, then do a count
passing_math_by_school = students_passing_math.groupby("school_name")

school_students_passing_math = passing_math_by_school["math_score"].count()

school_students_passing_math



school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: math_score, dtype: int64

In [106]:
#Calculate the number of students per school with math scores of 70 or higher
#duplicate the math section!

#loc to find scores > 70
students_passing_reading = merged_df.loc[merged_df["reading_score"] >= 70, :]

#group them by school, then do a count
passing_reading_by_school = students_passing_reading.groupby("school_name")

school_students_passing_reading = passing_reading_by_school["math_score"].count()

school_students_passing_reading



school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: math_score, dtype: int64

In [107]:
#calculate the number os students per school that passed both math and reading (70 or higher)
students_passing_math_and_reading = merged_df[
    (merged_df["reading_score"] >= 70) & (merged_df["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

school_students_passing_math_and_reading


school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [120]:
# Use the provided code to calculate the passing rates
#this is now working!
per_school_passing_math = school_students_passing_math / school_size * 100
per_school_passing_reading = school_students_passing_reading / school_size * 100
overall_passing_rate = school_students_passing_math_and_reading / school_size * 100

per_school_passing_math = per_school_passing_math.astype(float)
per_school_passing_reading = per_school_passing_reading.astype(float)
overall_passing_rate = overall_passing_rate.astype(float)





In [161]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({"School Type": school_types,
                                  "Total Students": school_size,
                                  "Total School Budget": per_school_budget,
                                  "Per Student Budget": per_school_capita,
                                  "Average Math Score": per_school_math,
                                  "Average Reading Score": per_school_reading,
                                   "% Passing Math": per_school_passing_math,
                                   "% Passing Reading": per_school_passing_reading,
                                   "% Overall Passing": overall_passing_rate
                                  })

school_size = school_size.astype(float)
per_school_budget = per_school_budget.astype(float)

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary["Average Math Score"] = per_school_summary["Average Math Score"].map("{:,.2f}".format)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].map("{:,.2f}".format)
per_school_summary["% Passing Math"] = per_school_summary["% Passing Math"].map("{:,.2f}%".format)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].map("{:,.2f}%".format)
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].map("{:,.2f}%".format)

# Minor data wrangling
per_school_summary.index.name = None

# Display the DataFrame
per_school_summary



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,[District],"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,[Charter],"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,[District],"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,[District],"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,[Charter],"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,[District],"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,[Charter],427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,[District],"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,[District],"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%
Pena High School,[Charter],962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


# Highest-Performing Schools (by % Overall Passing)

In [162]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,[Charter],"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,[Charter],"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,[Charter],"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,[Charter],"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,[Charter],962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


# Bottom Performing Schools (by % Overall Passing)

In [163]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,[District],"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,[District],"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,[District],"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,[District],"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,[District],"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


# Math Scores by Grade

In [164]:
# Use the code provided to separate the data by grade
ninth_graders = merged_df[(merged_df["grade"] == "9th")]
tenth_graders = merged_df[(merged_df["grade"] == "10th")]
eleventh_graders = merged_df[(merged_df["grade"] == "11th")]
twelfth_graders = merged_df[(merged_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_graders = ninth_graders.set_index(["school_name"])
ninth_graders = ninth_graders.groupby(["school_name"])

tenth_graders = tenth_graders.set_index(["school_name"])
tenth_graders = tenth_graders.groupby(["school_name"])

eleventh_graders = eleventh_graders.set_index(["school_name"])
eleventh_graders = eleventh_graders.groupby(["school_name"])

twelfth_graders = twelfth_graders.set_index(["school_name"])
twelfth_graders = twelfth_graders.groupby(["school_name"])

ninth_grade_math_scores = ninth_graders["math_score"].mean()
tenth_grade_math_scores = tenth_graders["math_score"].mean()
eleventh_grade_math_scores = eleventh_graders["math_score"].mean()
twelfth_grade_math_scores = twelfth_graders["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"Avg 9th Grade Math": ninth_grade_math_scores,
                                    "Avg 10th Grade Math": tenth_grade_math_scores,
                                    "Avg 11th Grade Math": eleventh_grade_math_scores,
                                    "Avg 12th Grade Math": twelfth_grade_math_scores})

# Minor data wrangling
math_scores_by_grade.index.name = None

#format with two decimals
math_scores_by_grade["Avg 9th Grade Math"] = math_scores_by_grade["Avg 9th Grade Math"].map("{:,.2f}".format)
math_scores_by_grade["Avg 10th Grade Math"] = math_scores_by_grade["Avg 10th Grade Math"].map("{:,.2f}".format)
math_scores_by_grade["Avg 11th Grade Math"] = math_scores_by_grade["Avg 11th Grade Math"].map("{:,.2f}".format)
math_scores_by_grade["Avg 12th Grade Math"] = math_scores_by_grade["Avg 12th Grade Math"].map("{:,.2f}".format)

# Display the DataFrame
math_scores_by_grade

,Avg 9th Grade Math,Avg 10th Grade Math,Avg 11th Grade Math,Avg 12th Grade Math
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


# Reading Scores by Grade

In [165]:
#duplicate the math section!

# Use the code provided to separate the data by grade
ninth_graders = merged_df[(merged_df["grade"] == "9th")]
tenth_graders = merged_df[(merged_df["grade"] == "10th")]
eleventh_graders = merged_df[(merged_df["grade"] == "11th")]
twelfth_graders = merged_df[(merged_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_graders = ninth_graders.set_index(["school_name"])
ninth_graders = ninth_graders.groupby(["school_name"])

tenth_graders = tenth_graders.set_index(["school_name"])
tenth_graders = tenth_graders.groupby(["school_name"])

eleventh_graders = eleventh_graders.set_index(["school_name"])
eleventh_graders = eleventh_graders.groupby(["school_name"])

twelfth_graders = twelfth_graders.set_index(["school_name"])
twelfth_graders = twelfth_graders.groupby(["school_name"])

ninth_grade_reading_scores = ninth_graders["reading_score"].mean()
tenth_grade_reading_scores = tenth_graders["reading_score"].mean()
eleventh_grade_reading_scores = eleventh_graders["reading_score"].mean()
twelfth_grade_reading_scores = twelfth_graders["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"Avg 9th Grade Reading": ninth_grade_reading_scores,
                                    "Avg 10th Grade Reading": tenth_grade_reading_scores,
                                    "Avg 11th Grade Reading": eleventh_grade_reading_scores,
                                    "Avg 12th Grade Reading": twelfth_grade_reading_scores})

# Minor data wrangling
reading_scores_by_grade.index.name = None

#format with 2 decimals
reading_scores_by_grade["Avg 9th Grade Reading"] = reading_scores_by_grade["Avg 9th Grade Reading"].map("{:,.2f}".format)
reading_scores_by_grade["Avg 10th Grade Reading"] = reading_scores_by_grade["Avg 10th Grade Reading"].map("{:,.2f}".format)
reading_scores_by_grade["Avg 11th Grade Reading"] = reading_scores_by_grade["Avg 11th Grade Reading"].map("{:,.2f}".format)
reading_scores_by_grade["Avg 12th Grade Reading"] = reading_scores_by_grade["Avg 12th Grade Reading"].map("{:,.2f}".format)

# Display the DataFrame
reading_scores_by_grade

,Avg 9th Grade Reading,Avg 10th Grade Reading,Avg 11th Grade Reading,Avg 12th Grade Reading
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


# Scores by School Spending

In [200]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [201]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

#Tip from Amanda in Slack- per student budget needs to be set back to float
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].str.replace('$','')
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].astype(float)



In [202]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, 
                                                             labels=labels, include_lowest=True)

#running dtypes shows scores and passrates are objects- change them back to float
school_spending_df["Average Math Score"] = school_spending_df["Average Math Score"].astype(float)
school_spending_df["Average Reading Score"] = school_spending_df["Average Reading Score"].astype(float)

school_spending_df['% Passing Math'] = school_spending_df['% Passing Math'].str.replace('%','')
school_spending_df["% Passing Math"] = school_spending_df["% Passing Math"].astype(float)

school_spending_df['% Passing Reading'] = school_spending_df['% Passing Reading'].str.replace('%','')
school_spending_df["% Passing Reading"] = school_spending_df["% Passing Reading"].astype(float)

school_spending_df['% Overall Passing'] = school_spending_df['% Overall Passing'].str.replace('%','')
school_spending_df["% Overall Passing"] = school_spending_df["% Overall Passing"].astype(float)

school_spending_df



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,[District],"4,976","$3,124,928.00",628,77,81,67,82,55,$585-630
Cabrera High School,[Charter],"1,858","$1,081,356.00",582,83,84,94,97,91,<$585
Figueroa High School,[District],"2,949","$1,884,411.00",639,77,81,66,81,53,$630-645
Ford High School,[District],"2,739","$1,763,916.00",644,77,81,68,79,54,$630-645
Griffin High School,[Charter],"1,468","$917,500.00",625,83,84,93,97,91,$585-630
Hernandez High School,[District],"4,635","$3,022,020.00",652,77,81,67,81,54,$645-680
Holden High School,[Charter],427,"$248,087.00",581,84,84,93,96,89,<$585
Huang High School,[District],"2,917","$1,910,635.00",655,77,81,66,81,54,$645-680
Johnson High School,[District],"4,761","$3,094,650.00",650,77,81,66,81,54,$645-680
Pena High School,[Charter],962,"$585,858.00",609,84,84,95,96,91,$585-630


In [203]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [206]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score": spending_math_scores,
                                "Average Reading Score": spending_reading_scores,
                                "% Passing Math": spending_passing_math,
                                "% Passing Reading": spending_passing_reading,
                                "% Overall Passing": overall_passing_spending})

#format!
spending_summary["Average Math Score"] = spending_summary["Average Math Score"].map("{:,.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:,.2f}".format)
spending_summary["% Passing Math"] = spending_summary["% Passing Math"].map("{:,.2f}%".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:,.2f}%".format)
spending_summary["% Overall Passing"] = spending_summary["% Overall Passing"].map("{:,.2f}%".format)

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.45,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


# Scores by School Size

In [210]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [214]:
# Categorize the spending based on the bins
#make a copy- NOTE- all datatypes on per_school_summary are objects, except Total Students (float)
per_school_summary2 = per_school_summary.copy()

# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary2["School Size"] = pd.cut(per_school_summary2["Total Students"], size_bins, 
                                                             labels=labels, include_lowest=True)

#running dtypes shows scores and passrates are objects- change them back to float
per_school_summary2["Average Math Score"] = per_school_summary2["Average Math Score"].astype(float)
per_school_summary2["Average Reading Score"] = per_school_summary2["Average Reading Score"].astype(float)

per_school_summary2['% Passing Math'] = per_school_summary2['% Passing Math'].str.replace('%','')
per_school_summary2["% Passing Math"] = per_school_summary2["% Passing Math"].astype(float)

per_school_summary2['% Passing Reading'] = per_school_summary2['% Passing Reading'].str.replace('%','')
per_school_summary2["% Passing Reading"] = per_school_summary2["% Passing Reading"].astype(float)

per_school_summary2['% Overall Passing'] = per_school_summary2['% Overall Passing'].str.replace('%','')
per_school_summary2["% Overall Passing"] = per_school_summary2["% Overall Passing"].astype(float)

 
per_school_summary2


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Bailey High School,[District],"4,976","$3,124,928.00",$628.00,77,81,67,82,55,Large (2000-5000)
Cabrera High School,[Charter],"1,858","$1,081,356.00",$582.00,83,84,94,97,91,Medium (1000-2000)
Figueroa High School,[District],"2,949","$1,884,411.00",$639.00,77,81,66,81,53,Large (2000-5000)
Ford High School,[District],"2,739","$1,763,916.00",$644.00,77,81,68,79,54,Large (2000-5000)
Griffin High School,[Charter],"1,468","$917,500.00",$625.00,83,84,93,97,91,Medium (1000-2000)
Hernandez High School,[District],"4,635","$3,022,020.00",$652.00,77,81,67,81,54,Large (2000-5000)
Holden High School,[Charter],427,"$248,087.00",$581.00,84,84,93,96,89,Small (<1000)
Huang High School,[District],"2,917","$1,910,635.00",$655.00,77,81,66,81,54,Large (2000-5000)
Johnson High School,[District],"4,761","$3,094,650.00",$650.00,77,81,66,81,54,Large (2000-5000)
Pena High School,[Charter],962,"$585,858.00",$609.00,84,84,95,96,91,Small (<1000)


In [215]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary2.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary2.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary2.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary2.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary2.groupby(["School Size"])["% Overall Passing"].mean()

In [217]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame ({"Average Math Score": size_math_scores,
                             "Average Reading Score": size_reading_scores,
                             "% Passing Math": size_passing_math,
                             "% Passing Reading": size_passing_reading,
                             "% Overall Passing": size_overall_passing})
#format!
size_summary["Average Math Score"] = size_summary["Average Math Score"].map("{:,.2f}".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:,.2f}".format)
size_summary["% Passing Math"] = size_summary["% Passing Math"].map("{:,.2f}%".format)
size_summary["% Passing Reading"] = size_summary["% Passing Reading"].map("{:,.2f}%".format)
size_summary["% Overall Passing"] = size_summary["% Overall Passing"].map("{:,.2f}%".format)

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.89%
Medium (1000-2000),83.37,83.87,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.28%


# Scores by School Type

In [230]:
#make a copy of the per_school_summary and reset the data types
per_school_summary3 = per_school_summary.copy()

per_school_summary3["Average Math Score"] = per_school_summary3["Average Math Score"].astype(float)
per_school_summary3["Average Reading Score"] = per_school_summary3["Average Reading Score"].astype(float)

per_school_summary3['% Passing Math'] = per_school_summary3['% Passing Math'].str.replace('%','')
per_school_summary3["% Passing Math"] = per_school_summary3["% Passing Math"].astype(float)

per_school_summary3['% Passing Reading'] = per_school_summary3['% Passing Reading'].str.replace('%','')
per_school_summary3["% Passing Reading"] = per_school_summary3["% Passing Reading"].astype(float)

per_school_summary3['% Overall Passing'] = per_school_summary3['% Overall Passing'].str.replace('%','')
per_school_summary3["% Overall Passing"] = per_school_summary3["% Overall Passing"].astype(float)

per_school_summary3["School Type"] = per_school_summary3["School Type"].astype(str)

#display
per_school_summary3.dtypes





School Type               object
Total Students           float64
Total School Budget       object
Per Student Budget        object
Average Math Score       float64
Average Reading Score    float64
% Passing Math           float64
% Passing Reading        float64
% Overall Passing        float64
dtype: object

In [232]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary3.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary3.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary3.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary3.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary3.groupby(["School Type"])["% Overall Passing"].mean()

In [233]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame ({"Average Math Score": average_math_score_by_type,
                             "Average Reading Score": average_reading_score_by_type,
                             "% Passing Math": average_percent_passing_math_by_type,
                             "% Passing Reading": average_percent_passing_reading_by_type,
                             "% Overall Passing": average_percent_overall_passing_by_type})
#format!
type_summary["Average Math Score"] = type_summary["Average Math Score"].map("{:,.2f}".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:,.2f}".format)
type_summary["% Passing Math"] = type_summary["% Passing Math"].map("{:,.2f}%".format)
type_summary["% Passing Reading"] = type_summary["% Passing Reading"].map("{:,.2f}%".format)
type_summary["% Overall Passing"] = type_summary["% Overall Passing"].map("{:,.2f}%".format)

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
['Charter'],83.47,83.90,93.62%,96.59%,90.43%
['District'],76.96,80.97,66.55%,80.80%,53.67%
